In [ ]:
from contextlib import redirect_stdout
from gurobipy import Model, GRB
import data


# Parametros
C_ij = {(i, j): data.costos_apertura[(i, j)] for i in data.ciudades for j in data.tipos_planta}  # Costo de apertura
Ctikf = {(i, k, f): data.costos_transporte[(f, i, k)] for i in data.ciudades for k in data.regiones for f in data.transportes}  # Costo de transporte
Dk_actual = {k: data.demanda_actual[k] for k in data.regiones}  # Demanda actual
T_k = {k: data.tasas_crecimiento[k] for k in data.regiones}  # Tasa de crecimiento
Dk_t = {(k, t): Dk_actual[k] * (1 + T_k[k])**t for k in data.regiones for t in data.años}  # Demanda futura
Cpp = {j: data.capacidad_planta[j] for j in data.tipos_planta}  # Capacidad de producción
Cv_ij = {(i, j): data.costos_variables[(i, j)] for i in data.ciudades for j in data.tipos_planta}  # Costo variable
Cfij = {(i, j): data.costos_fijos[(i, j)] for i in data.ciudades for j in data.tipos_planta}  # Costo fijo


# Verificacion
'''
print('Costo de transporte: ',C_ij)
print('Demanda actual: ',Dk_actual)
print('Tasa de crecimiento: ',T_k)
print('Demanda futura: ',Dk_t)
print('Capacidad de producción: ',Cpp)
print('Costo variable: ',Cv_ij)
print('Costo fijo: ',Cfij)
'''

# Modelo
model = Model("Optimización de Produccion")


# Variables
X_ij = model.addVars(data.ciudades, data.tipos_planta, vtype=GRB.BINARY, name="X")
Y_ikft = model.addVars(data.ciudades, data.regiones, data.transportes, data.años, vtype=GRB.CONTINUOUS, name="Y")


# Restricciones
for j in data.tipos_planta:
    model.addConstr(sum(X_ij[i, j] for i in data.ciudades) <= 1)

for i in data.ciudades:
    model.addConstr(sum(X_ij[i, j] for j in data.tipos_planta) <= 1)


for t in data.años:
    for j in data.tipos_planta:
        model.addConstr(sum(Y_ikft[i, k, f, t] for i in data.ciudades for k in data.regiones for f in data.transportes) <= sum(X_ij[i, j] * Cpp[j] for i in data.ciudades))


D_unmet = model.addVars(data.regiones, data.años, vtype=GRB.CONTINUOUS, name="D_unmet")
for k in data.regiones:
    for t in data.años:
        model.addConstr(sum(Y_ikft[i, k, f, t] for i in data.ciudades for f in data.transportes) + D_unmet[k, t] >= Dk_t[k, t])






# Función objetivo (funcion 1 del word)
alpha = 0.1  # Factor de reducción de costos fijos
model.setObjective(
    sum(alpha * (C_ij[i, j] + Cfij[i, j]) * X_ij[i, j] for i in data.ciudades for j in data.tipos_planta) +
    sum(Cv_ij[i, j] * X_ij[i, j] for i in data.ciudades for j in data.tipos_planta) +
    sum(Ctikf[i, k, f] * Y_ikft[i, k, f, t] for i in data.ciudades for k in data.regiones for f in data.transportes for t in data.años),
    GRB.MINIMIZE
)



# Función objetivo (funcion 2 del word)
'''
model.setObjective(
    sum(C_ij[i, j] * X_ij[i, j] for i in data.ciudades for j in data.tipos_planta) +
    sum(Cfij[i, j] * X_ij[i, j] for i in data.ciudades for j in data.tipos_planta) +
    sum((Cv_ij[i, j] + Ctikf[i, k, f]) * Y_ikft[i, k, f, t] 
        for i in data.ciudades for j in data.tipos_planta 
        for k in data.regiones for f in data.transportes for t in data.años),
    GRB.MINIMIZE
)
'''



# Optimizando
log = "model.txt"
with open(log, "w", encoding="utf-8") as file:
    with redirect_stdout(file):
        #model.update()
        model.optimize()



# Ver data
for constr in model.getConstrs():
    if constr.Slack < 0:
        print(f"{constr.ConstrName}: Slack = {constr.Slack}")
print("Demanda futura (Dk_t):", Dk_t)
print("Capacidades de producción (Cpp):", Cpp)
for i in data.ciudades:
    for j in data.tipos_planta:
        print(f"X_ij[{i},{j}] = {X_ij[i, j].X}")



# Resultados
with open('results.txt', 'w', encoding='utf-8') as arch:
    arch.write(f'Función objetivo: {model.objVal}\n')
    print(f'Función objetivo: {model.objVal}')

    if model.status == GRB.OPTIMAL:
        for c in data.ciudades:
            for r in data.regiones:
                for t in data.transportes:
                    for an in data.años:
                        if (c, r, t, an) in Y_ikft:
                            valor = Y_ikft[c, r, t, an].X
                            arch.write(f"Y_ikft[{c}, {r}, {t}, {an}] = {valor}\n")
                            print(f"Y_ikft[{c}, {r}, {t}, {an}] = {valor}")



R13: Slack = -98895939.84
R14: Slack = -98719290.2144
R15: Slack = -98514376.648704
R16: Slack = -98819815.92
R17: Slack = -98560175.4224
R18: Slack = -98243414.015328
R19: Slack = -99354815.74
R20: Slack = -99187067.8324
R21: Slack = -98975705.468824
R22: Slack = -99555814.8
R23: Slack = -99489187.02
R24: Slack = -99412565.073
R25: Slack = -98684818.14
R26: Slack = -98171897.2146
R27: Slack = -97458937.128294
R28: Slack = -99605521.5
R29: Slack = -99487177.95
R30: Slack = -99333331.335
Demanda futura (Dk_t): {('R1', 1): 1104060.16, ('R1', 2): 1280709.7855999998, ('R1', 3): 1485623.3512959997, ('R2', 1): 1180184.08, ('R2', 2): 1439824.5776, ('R2', 3): 1756585.984672, ('R3', 1): 645184.26, ('R3', 2): 812932.1676, ('R3', 3): 1024294.5311760001, ('R4', 1): 444185.19999999995, ('R4', 2): 510812.9799999999, ('R4', 3): 587434.9269999999, ('R5', 1): 1315181.86, ('R5', 2): 1828102.7854000004, ('R5', 3): 2541062.871706001, ('R6', 1): 394478.5, ('R6', 2): 512822.05000000005, ('R6', 3): 666668.66

OTRO

In [5]:
from gurobipy import Model, GRB
from contextlib import redirect_stdout
import pandas as pd
import data



# Parámetros
C_ij = {(i, j): data.costos_apertura[(i, j)] for i in data.ciudades for j in data.tipos_planta}  # Costo de apertura
Ctikf = {(i, k, f): data.costos_transporte[(f, i, k)] for i in data.ciudades for k in data.regiones for f in data.transportes}  # Costo de transporte
Dk_actual = {k: data.demanda_actual[k] for k in data.regiones}  # Demanda actual
T_k = {k: data.tasas_crecimiento[k] for k in data.regiones}  # Tasa de crecimiento
Dk_t = {(k, t): Dk_actual[k] * (1 + T_k[k])**t for k in data.regiones for t in data.años}  # Demanda futura
Cpp = {j: data.capacidad_planta[j] for j in data.tipos_planta}  # Capacidad de producción
Cv_ij = {(i, j): data.costos_variables[(i, j)] for i in data.ciudades for j in data.tipos_planta}  # Costo variable
Cfij = {(i, j): data.costos_fijos[(i, j)] for i in data.ciudades for j in data.tipos_planta}  # Costo fijo


# Modelo
model = Model("Optimización de Producción")

# Variables
X_ij = model.addVars(data.ciudades, data.tipos_planta, vtype=GRB.BINARY, name="X")
Y_ikft = model.addVars(data.ciudades, data.regiones, data.transportes, data.años, vtype=GRB.CONTINUOUS, name="Y")

# Restricciones
for j in data.tipos_planta:
    model.addConstr(sum(X_ij[i, j] for i in data.ciudades) >= 1)

for i in data.ciudades:
    model.addConstr(sum(X_ij[i, j] for j in data.tipos_planta) >= 1)


for i in data.ciudades:
    for t in data.años:
        for j in data.tipos_planta:
            model.addConstr(sum(Y_ikft[i, k, f, t] for k in data.regiones for f in data.transportes) <= sum(X_ij[i, j] * Cpp[j] for i in data.ciudades))

for k in data.regiones:
    for t in data.años:
        model.addConstr(sum(Y_ikft[i, k, f, t] for i in data.ciudades for f in data.transportes) >= Dk_t[k, t])

# Función objetivo (funcion 2 del word)
model.setObjective(
    sum(C_ij[i, j] * X_ij[i, j] for i in data.ciudades for j in data.tipos_planta) +
    sum(Cfij[i, j] * X_ij[i, j] for i in data.ciudades for j in data.tipos_planta) +
    sum((Cv_ij[i, j] + Ctikf[i, k, f]) * Y_ikft[i, k, f, t] 
        for i in data.ciudades for j in data.tipos_planta 
        for k in data.regiones for f in data.transportes for t in data.años),
    GRB.MINIMIZE
)

# Optimizando
log = "model.txt"
with open(log, "w", encoding="utf-8") as file:
    with redirect_stdout(file):
        #model.update()
        model.optimize()




# Resultados
arch = open('results.txt', 'w', encoding='utf-8')
arch.write('Función objetivo: ' + str(model.objVal) + '\n')
print('Función objetivo: ', model.objVal)

if model.status == GRB.OPTIMAL:
    for c in data.ciudades:
        for r in data.regiones:
            for t in data.transportes:
                for an in data.años:
                    if Y_ikft[c, r, t, an].X > 0:
                        print(f"Transportar {Y_ikft[c, r, t, an].X} unidades desde {c} a {r} usando {t} en el año {an}")
                        arch.write(f"Transportar {Y_ikft[c, r, t, an].X} unidades desde {c} a {r} usando {t} en el año {an}\n")

arch.close()


Función objetivo:  1655576250.1431863
Transportar 1104060.16 unidades desde Santiago a R1 usando AT2 en el año 1
Transportar 1280709.7855999998 unidades desde Santiago a R1 usando AT2 en el año 2
Transportar 1485623.3512959997 unidades desde Santiago a R1 usando AT2 en el año 3
Transportar 1180184.08 unidades desde Santiago a R2 usando AT1 en el año 1
Transportar 1439824.5776 unidades desde Santiago a R2 usando AT1 en el año 2
Transportar 1756585.984672 unidades desde Santiago a R2 usando AT1 en el año 3
Transportar 645184.26 unidades desde Santiago a R3 usando AT1 en el año 1
Transportar 812932.1676 unidades desde Santiago a R3 usando AT1 en el año 2
Transportar 1024294.5311760001 unidades desde Santiago a R3 usando AT1 en el año 3
Transportar 444185.19999999995 unidades desde Santiago a R4 usando AT1 en el año 1
Transportar 510812.9799999999 unidades desde Santiago a R4 usando AT1 en el año 2
Transportar 587434.9269999999 unidades desde Santiago a R4 usando AT1 en el año 3
Transporta